# MCTS POMDP

Reusing much of polar_pomdp0.45 for problem structure, particle filter, etc.

### Setup

In [1]:
using Plots
using ParticleFilters
using Distributions
using StaticArrays
using LinearAlgebra
using Random
using StatsBase
#using Reel
using SparseArrays
using GridInterpolations
using DataStructures
using DataFrames
using CSV
using Distributed

┌ Info: Recompiling stale cache file C:\Users\T450S\.julia\compiled\v1.2\Plots\ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file C:\Users\T450S\.julia\compiled\v1.2\ParticleFilters\vVum1.ji for ParticleFilters [c8b314e2-9260-5cf8-ae76-3be7461ca6d0]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file C:\Users\T450S\.julia\compiled\v1.2\GridInterpolations\7sRrY.ji for GridInterpolations [bb4c363b-b914-514b-8517-4eb369bc008a]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file C:\Users\T450S\.julia\compiled\v1.2\DataFrames\AR9oZ.ji for DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file C:\Users\T450S\.julia\compiled\v1.2\CSV\HHBkp.ji for CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1240


In [25]:
include("atan2.jl")
include("obsv2.jl") # includes functions for generating observations for problem POMDP
include("rules.jl") # rules-based action
;

In [6]:
rng = MersenneTwister(2)
TGT_SPD = 1
;

### Problem Structure

Function to randomly determine next target course

In [7]:
function next_crs(crs,rng)
    if rand(rng) < .9
        return crs
    end
    crs = (crs + rand(rng,[-1,1])*30) % 360
    if crs < 0 crs += 360 end
    return crs
end
;

True state transition function

In [8]:
# state as tuple (x, y, crs, spd) of target (spd of o/s)
function f(state, control, rng)
    r, θ, crs, spd = state
    θ += control[1]
    spd = control[2]
    if θ < 0 θ += 360 end
    θ = θ % 360
    crs -= control[1]
    if crs < 0 crs += 360 end
    crs = crs % 360
    x = r*cos(π/180*θ)
    y = r*sin(π/180*θ)
    pos = [x + TGT_SPD*cos(π/180*crs) - spd, y + 
        TGT_SPD*sin(π/180*crs)]
    crs = next_crs(crs,rng)
    r = sqrt(pos[1]^2 + pos[2]^2)
    θ = atan2(pos[1],pos[2])*180/π
    if θ < 0 θ += 360 end
    return (r, θ, crs, spd)::NTuple{4, Real}
end
;

Wrapper for f that returns vector rather than Tuple for particle filter

In [9]:
function f2(x, u, rng)
    temp = [i for i in f(x, u, rng)]
    return temp
end
;

Reward function

In [60]:
function r(s)
    range = s[1]
    if range > 250 return [0, -.1, 0] end  # reward to not lose track of contact
    if range <= 10 return [-.1, 0, 0] end  # collision avoidance
    return [0, 0, .1]  # being in "sweet spot" maximizes reward
end

function abs_r(reward::Array{Float64,1})
    return reward[1] + reward[2] + reward[3]
end

function event_r(reward::Array{Float64,1})
    return [1*(reward[1] < 0), 1*(reward[2] < 0)]
end
;

Action space and function to convert from action to index and vice versa

In [11]:
action_space = ((-30,1), (-30, 2), (0, 1), (0, 2), (30, 1), (30, 2))

action_to_index(a) = trunc(Int, 2*(a[1]/30+1) + a[2])

function index_to_action(a)
    if a % 2 == 0
        return ( trunc(Int,(((a - 2) / 2) - 1) * 30), 2)
    else
        return ( trunc(Int,(((a - 1) / 2) - 1) * 30), 1)
    end
end
;

6

### Particle Filter

Will be used for our belief state

In [12]:
num_particles = 500
model = ParticleFilterModel{Vector{Float64}}(f2, g)
pfilter = SIRParticleFilter(model, num_particles)
;

## MCTS Algorithm

#### MCTS Functions

Function to return index of optimal action using current Q values and possibly the exploration bonus

In [13]:
function arg_max_action(Q, N, history, c=nothing, exploration_bonus=false)
    
    # only need to compute if exploration possibility
    if exploration_bonus
        N_h = 0
        for action in action_to_index.(action_space)
            new_index = copy(history)
            append!(new_index, action)
            N_h += N[new_index]
        end    
    end
    
    values = Float64[]
    for action in action_to_index.(action_space)
        
        new_index = copy(history)
        append!(new_index, action)
        
        # best action with exploration possibility
        if exploration_bonus
            
            # ensure an action chosen zero times is always chosen
            if N[new_index] == 0
                return action
            end
            
            # compute exploration bonus, checking for zeroes (I don't think this will ever occur anyway...)
            if log(N_h) < 0
                numerator = 0
            else
                numerator = sqrt(log(N_h))
            end
            denominator = N[new_index]
            exp_bonus = c * numerator / denominator
            append!(values, Q[new_index] + exp_bonus)
        
        # strictly best action
        else
            append!(values, Q[new_index])
        end
    end
    
    return argmax(values)
    
end
;

Function to rollout with random actions until we reach satisfactory depth

In [14]:
function rollout_random(state, depth)
    
    if depth == 0 return 0 end
    
    # random action
    random_action_index = rand(rng,action_to_index.(action_space))
    action = index_to_action(random_action_index)
    
    # generate next state and reward with random action; observation doesn't matter
    state_prime = f2(state, action, rng)
    reward = abs_r(r(Tuple(state_prime)))
    
    return reward + lambda * rollout_random(state_prime, depth-1)
    
end
;

Simulate function includes search, expansion, and rollout

In [15]:
function simulate(Q, N, state, history, depth, c)
    
    if depth == 0 return (Q, N, 0) end
    
    
    # expansion
    test_index = copy(history)
    append!(test_index, 1)
    
    if !haskey(Q, test_index)
        
        for action in action_to_index.(action_space)
            # initialize Q and N to zeros
            new_index = copy(history)
            append!(new_index, action)
            Q[new_index] = 0
            N[new_index] = 0        
        end

        # rollout
        return (Q, N, rollout_random(state, depth))
        
    end
    
    
    # search
    # find optimal action to explore
    search_action_index = arg_max_action(Q, N, history, c, true)
    action = index_to_action(search_action_index)
    
    # take action; get new state, observation, and reward
    state_prime = f2(state, action, rng)
    observation = h(state_prime, rng)
    reward = abs_r(r(Tuple(state_prime)))
    
    # recursive call after taking action and getting observation
    new_history = copy(history)
    append!(new_history, search_action_index)
    append!(new_history, observation)
    (Q, N, successor_reward) = simulate(Q, N, state_prime, new_history, depth-1, c)
    q = reward + lambda * successor_reward
    
    # update counts and values
    update_index = copy(history)
    append!(update_index, search_action_index)
    N[update_index] += 1
    Q[update_index] += ((q - Q[update_index]) / N[update_index])
    
    return (Q, N, q)
    
end
;

Main MCTS function; called by MCTS wrapper at each time step to choose an action

In [17]:
function select_action(Q, N, belief, depth, c)
    
    # empty history at top recursive call
    history = Int64[]
    
    # loop
    # timed loop, how long should intervals be?
    #counter = 0
    #start_time = time_ns()
    #while (time_ns() - start_time) / 1.0e9 < 1 # 1 second timer to start
    
    # counted iterations for now, would switch to time for a production model
    counter = 0
    while counter < 100 # probably increase; small for debugging
        
        # draw state randomly based on belief state (pick a random particle)
        state = rand(rng,belief)
        
        # simulate
        simulate(Q, N, float(state), history, depth, c)
        
        counter+=1
    end
    #println(counter, " iterations")
    
    best_action_index = arg_max_action(Q, N, history)
    action = index_to_action(best_action_index)
    return (Q, N, action)
    
end
;

#### MCTS simulation

Function to advance history tree after an action is chosen and observation is recorded

NOTE: We ended up not using this function. With our problem structure, we found that restarting the MCTS algorithm at every time step yielded better results.

In [18]:
function modify_history_tree(Q, N, last_action, last_obs)
    
    newQ = Dict{Array{Int64,1},Float64}()
    newN = Dict{Array{Int64,1},Float64}()
    
    for key in keys(Q)
        # if key matches last action and observation, becomes root in new tree
        if length(key) > 2 && key[1] == action_to_index(last_action) && key[2] == last_obs
            newQ[key[3:length(key)]] = Q[key]
            newN[key[3:length(key)]] = N[key]
        else
            continue
        end
    end
    
    return (newQ, newN)
    
end
;

Perform a 500 time step trial with MCTS

In [46]:
# global scope params (that are not being experimented with)
lambda = 0.95
;

In [56]:
function trial(depth, c)
    
    # Initialize true state and belief state (particle filter); we assume perfect knowledge at start of simulation (could experiment otherwise with random beliefs)
    
    # true state
    # for now state is [range, bearing, relative course, own speed]
    # assume a starting position within range of sensor and not too close
    true_state = [rand(rng, 25:100), rand(rng,0:359), rand(rng,0:11)*30, 1]

    # belief state
    # assume perfect knowledge at first time step
    #belief = ParticleCollection([true_state for i in 1:num_particles])
    # assume we just know it's between 25-100 distance
    belief = ParticleCollection([[rand(rng, 25:100), rand(rng,0:359), rand(rng,0:11)*30, 1] for i in 1:num_particles])
    
    
    
    # Simulation prep/initialization; for now we start with no prior knowledge for Q values/N values, could incorporate this later
    
    # global Q and N dictionaries, indexed by history (and optionally action to follow all in same array; using ints)
    Q = Dict{Array{Int64,1},Float64}()
    N = Dict{Array{Int64,1},Float64}()

    # not manipulating these parameters for now, in global scope
    # lambda, discount factor
    #lambda = 0.95

    # experimenting with different parameter values
    # experiment with different depth parameters 
    depth = depth
    # exploration factor, experiment with different values
    c = c
    
    # don't need to modify history tree at first time step
    action = nothing
    observation = nothing
    
    
    
    # run simulation
    
    total_reward = [0, 0, 0]
    
    # 500 time steps with an action to be selected at each
    num_iters = 500
    
    for time_step = 1:num_iters
       
        #if time_step % 100 == 0 
        #    @show time_step
        #end
    
        # NOTE: we found restarting history tree at each time step yielded better results
        # if action taken, modify history tree
        if action != nothing
            #(Q,N) = modify_history_tree(Q, N, action, observation)
            Q = Dict{Array{Int64,1},Float64}()
            N = Dict{Array{Int64,1},Float64}()
        end
    
    
        # select an action
        (Q, N, action) = select_action(Q, N, belief, depth, c)
    
        # take action; get next true state, obs, and reward
        next_state = f2(true_state, action, rng)
        observation = h(next_state, rng)
        reward = abs_r(r(Tuple(next_state)))
        true_state = next_state
    
        # update belief state (particle filter)
        belief = update(pfilter, belief, action, observation)
    
        # accumulate reward
        total_reward += r(Tuple(next_state))
        
        # TODO: flags for collision, lost track, end of simulation lost track
    
    end
    
    return total_reward
    
end
;

#### Parameter Optimization

Try different depths (use c=20, 100 iterations per select action as pre-optimization defaults)

An aside, but we switched the loop in the select action function to a timed loop, and doing so we saw that we were able to perform roughly 60,000 iterations per second for depth 1, 9,000 for depth 5, 7,000 for depth 10, 5,500 for depth 25, 4,000 for depth 50, and 2,500 for depth 100.

We would expect to see better results with more iterations, but there is a tradeoff with time. We will use 100 iterations for parameter optimization strictly for timing purposes, but ideally we would use more iterations.

In [58]:
# collect total reward for each trial
#mcts_depth1_reward = Float64[]
#mcts_depth5_reward = Float64[]
mcts_depth10_reward = [0, 0]
mcts_depth25_reward = [0, 0]
#mcts_depth50_reward = Float64[]
#mcts_depth100_reward = Float64[]

# trials
for i = 1:20
    
    #append!(mcts_depth1_reward, trial(1, 20))
    #append!(mcts_depth5_reward, trial(5, 20))
    mcts_depth10_reward += event_r(trial(10, 20))
    mcts_depth25_reward += event_r(trial(25, 20))
    #append!(mcts_depth50_reward, trial(50, 20))
    #append!(mcts_depth100_reward, trial(100, 20))
    
    if i % 1 == 0
        println(i, " runs complete")
    end
end

# print results

#println("")
#println("MCTS depth 1 reward")
#println(mcts_depth1_reward)
#println("Average: ", mean(mcts_depth1_reward))


#println("")
#println("MCTS depth 5 reward")
#println(mcts_depth5_reward)
#println("Average: ", mean(mcts_depth5_reward))

println("")
println("MCTS depth 10 reward")
println(mcts_depth10_reward)
println("Average: ", mean(mcts_depth10_reward))

println("")
println("MCTS depth 25 reward")
println(mcts_depth25_reward)
println("Average: ", mean(mcts_depth25_reward))

#println("")
#println("MCTS depth 50 reward")
#println(mcts_depth50_reward)
#println("Average: ", mean(mcts_depth50_reward))

#println("")
#println("MCTS depth 100 reward")
#println(mcts_depth100_reward)
#println("Average: ", mean(mcts_depth100_reward))

1 runs complete
2 runs complete
3 runs complete
4 runs complete
5 runs complete
6 runs complete
7 runs complete
8 runs complete
9 runs complete
10 runs complete
11 runs complete
12 runs complete
13 runs complete
14 runs complete
15 runs complete
16 runs complete
17 runs complete
18 runs complete
19 runs complete
20 runs complete

MCTS depth 10 reward
[6, 0]
Average: 3.0

MCTS depth 25 reward
[1, 0]
Average: 0.5


It appears the depth doesn't really matter as we had only one non-max reward.  Obviously then, we would favor a smaller depth as it will run through iterations faster.  We will make sure of our results by running a hundred trials for less time intensive depths:

In [ ]:
# collect total reward for each trial
mcts_depth1_reward = Float64[]
mcts_depth5_reward = Float64[]
mcts_depth10_reward = Float64[]

# trials
for i = 1:100
    
    append!(mcts_depth1_reward, trial(1, 20))
    append!(mcts_depth5_reward, trial(5, 20))
    append!(mcts_depth10_reward, trial(10, 20))
    
    if i % 10 == 0
        println(i, " runs complete")
    end
end

# print results
println("")
println("MCTS depth 1 reward")
println(mcts_depth1_reward)
println("Average: ", mean(mcts_depth1_reward))

println("")
println("MCTS depth 5 reward")
println(mcts_depth5_reward)
println("Average: ", mean(mcts_depth5_reward))

println("")
println("MCTS depth 10 reward")
println(mcts_depth10_reward)
println("Average: ", mean(mcts_depth10_reward))

It is a bit curious why depth 10 underperforms (perhaps it gets too complicated the further we extrapolate; maybe we should be discounting more).  However, 1 and 5 appear to work perfectly.  Just to be certain, we'll make sure we always get maximum reward (using 1, which is the fastest):

In [ ]:
# collect total reward for each trial
mcts_depth1_reward = Float64[]

# trials
for i = 1:1000
    
    append!(mcts_depth1_reward, trial(1, 20))
    
    if i % 100 == 0
        println(i, " runs complete")
    end
end

# print results
println("")
println("MCTS depth 1 reward")
println(mcts_depth1_reward)
println("Average: ", mean(mcts_depth1_reward))

Over a thousand trials, we always achieve the max reward with a depth of 1.  We time a single trial to for real world application purposes:

In [ ]:
start_time = time_ns()
println(trial(1, 20))
println((time_ns() - start_time) / 1.0e9)

We select 500 actions in 1.52378689 seconds, or just over 328 actions per second.

#### Random Trial

We compare MCTS performance to random action selection.

In [21]:
function random_trial()
    
    # Initialize true state and belief state (particle filter); we assume perfect knowledge at start of simulation (could experiment otherwise with random beliefs)
    
    # true state
    # for now state is [range, bearing, relative course, own speed]
    # assume a starting position within range of sensor and not too close
    true_state = [rand(rng, 100:150), rand(rng,0:359), rand(rng,0:11)*30, 1]

    # belief state
    # assume perfect knowledge at first time step
    belief = ParticleCollection([true_state for i in 1:num_particles])
    
    
    
    # run simulation
    
    total_reward = [0, 0, 0]

    # 500 time steps with an action to be selected at each
    num_iters = 500
    
    for time_step = 1:num_iters
    
        #if time_step % 100 == 0 
        #    @show time_step
        #end
    
        action = rand(rng, action_space)
    
        # take action; get next true state, obs, and reward
        next_state = f2(true_state, action, rng)
        observation = h(next_state, rng)
        reward = abs_r(r(Tuple(next_state)))
        true_state = next_state
    
        # accumulate reward
        total_reward += r(Tuple(next_state))
    
    end
    
    return total_reward
    
end
;

#### Rules-based trials

In [55]:
function rules_trial(risk)
    
    # Initialize true state and belief state (particle filter); we assume perfect knowledge at start of simulation (could experiment otherwise with random beliefs)
    
    # true state
    # for now state is [range, bearing, relative course, own speed]
    # assume a starting position within range of sensor and not too close
    true_state = [rand(rng, 100:150), rand(rng,0:359), rand(rng,0:11)*30, 1]

    # belief state
    # assume perfect knowledge at first time step
    belief = ParticleCollection([true_state for i in 1:num_particles])
    
    
    
    # run simulation
    
    total_reward = [0, 0, 0]

    # 500 time steps with an action to be selected at each
    num_iters = 500
    
    for time_step = 1:num_iters
    
        #if time_step % 100 == 0 
        #    @show time_step
        #end
    
        action = index_to_action(rules_action(true_state, risk))
    
        # take action; get next true state, obs, and reward
        next_state = f2(true_state, action, rng)
        observation = h(next_state, rng)
        reward = abs_r(r(Tuple(next_state)))
        true_state = next_state
    
        # accumulate reward
        total_reward += r(Tuple(next_state))
    
    end
    
    return total_reward
    
end
;

Compare random reward to MCTS of various depths

rules_action (generic function with 2 methods)

In [230]:
include("rules.jl") # rules-based action
# collect total reward for each trial
rules_reward = [0, 0]

# trials
for i = 1:200
    
    rules_reward += event_r(rules_trial(0.8))
    
    if i % 10 == 0
        println(i, " runs complete")
    end
end

# print results
# [close, far, good]
println("")
print("Result: ")
println(rules_reward)

10 runs complete
20 runs complete
30 runs complete
40 runs complete
50 runs complete
60 runs complete
70 runs complete
80 runs complete
90 runs complete
100 runs complete
110 runs complete
120 runs complete
130 runs complete
140 runs complete
150 runs complete
160 runs complete
170 runs complete
180 runs complete
190 runs complete
200 runs complete

Result: [0, 0]


The rules perform perfectly with perfect information. But we do not have perfect information.

In [219]:
function rules_trial2(risk)
    
    # Initialize true state and belief state (particle filter); we assume perfect knowledge at start of simulation (could experiment otherwise with random beliefs)
    
    # true state
    # for now state is [range, bearing, relative course, own speed]
    # assume a starting position within range of sensor and not too close
    true_state = [rand(rng, 100:150), rand(rng,0:359), rand(rng,0:11)*30, 1]

    # belief state
    # assume perfect knowledge at first time step
    belief = ParticleCollection([true_state for i in 1:num_particles])
    #print(mean(belief))
    
    
    # run simulation
    
    total_reward = [0, 0, 0]

    # 500 time steps with an action to be selected at each
    num_iters = 500
    
    for time_step = 1:num_iters
    
        #if time_step % 100 == 0 
        #    @show time_step
        #end
    
        action = index_to_action(rules_action(mean(belief), risk))
    
        # take action; get next true state, obs, and reward
        next_state = f2(true_state, action, rng)
        observation = h(next_state, rng)
        reward = abs_r(r(Tuple(next_state)))
        true_state = next_state
    
        # accumulate reward
        total_reward += r(Tuple(next_state))
    
    end
    
    return total_reward
    
end
;

In [236]:
include("rules.jl") # rules-based action
# collect total reward for each trial
rules_reward = [0, 0]

# trials
for i = 1:200
    
    rules_reward += event_r(rules_trial2(0.9))
    
    if i % 10 == 0
        println(i, " runs complete")
    end
end

# print results
# [close, far, good]
println("")
print("Result: ")
println(rules_reward)

10 runs complete
20 runs complete
30 runs complete
40 runs complete
50 runs complete
60 runs complete
70 runs complete
80 runs complete
90 runs complete
100 runs complete
110 runs complete
120 runs complete
130 runs complete
140 runs complete
150 runs complete
160 runs complete
170 runs complete
180 runs complete
190 runs complete
200 runs complete

Result: [3, 43]


#### Plot Results

In [ ]:
# no plots at this time

### Julia scratch space

2-element Array{Int64,1}:
 0
 0